In [1]:
from Text2SQL.SnowflakeCore import *

In [2]:
schema = get_schema_context("AIRLINES", "AIRLINES", model_name = model_name)
print(schema)

Reading Tables....


100%|██████████| 8/8 [00:09<00:00,  1.20s/it]


Building up FKs....
dialect='snowflake' database='AIRLINES' Schema='AIRLINES' tables=[TableAttr(name='AIRCRAFTS_DATA', column_names=[{'name': 'aircraft_code', 'type': 'VARCHAR(16777216)', 'null?': 'Y'}, {'name': 'model', 'type': 'VARCHAR(16777216)', 'null?': 'Y'}, {'name': 'range', 'type': 'NUMBER(38,0)', 'null?': 'Y'}], sample_rows=[{'aircraft_code': '773', 'model': '{"en": "Boeing 777-300", "ru": "Боинг 777-300"}', 'range': 11100}], row_count=9), TableAttr(name='AIRPORTS_DATA', column_names=[{'name': 'airport_code', 'type': 'VARCHAR(16777216)', 'null?': 'Y'}, {'name': 'airport_name', 'type': 'VARCHAR(16777216)', 'null?': 'Y'}, {'name': 'city', 'type': 'VARCHAR(16777216)', 'null?': 'Y'}, {'name': 'coordinates', 'type': 'VARCHAR(16777216)', 'null?': 'Y'}, {'name': 'timezone', 'type': 'VARCHAR(16777216)', 'null?': 'Y'}], sample_rows=[{'airport_code': 'YKS', 'airport_name': '{"en": "Yakutsk Airport", "ru": "Якутск"}', 'city': '{"en": "Yakutsk", "ru": "Якутск"}', 'coordinates': '(129.7709

In [3]:
def create_system_prompt(schema: ParentSchema) -> str:
    return f"""You are a SQL query generator for {schema.dialect}.

DATABASE: {schema.database}.{schema.Schema}
SCHEMA: {schema.model_dump_json(indent=2)}

CORE RULES:
1. Generate SELECT queries only - no modifications
2. ALWAYS quote identifiers: SELECT "column_name" FROM "table_name"
3. Use fully qualified names: {schema.database}.{schema.Schema}."TABLE_NAME"
4. No semicolons - single statement only
5. Use the execute_query tool to run queries

JSON COLUMNS:
- Many columns store JSON as VARCHAR (see samples)
- To extract values: GET(PARSE_JSON("column_name"), 'key')::VARCHAR
- Example: GET(PARSE_JSON("airport_name"), 'en')::VARCHAR

NULL VALUES:
- Sample data shows '\\N' but use IS NULL / IS NOT NULL in WHERE clauses

ERROR RECOVERY:
- If query fails, check: quoted identifiers, table/column names, JSON syntax
- Analyze error message and retry with corrections

Be precise and efficient."""
SYSTEM_PROMPT = create_system_prompt(schema)
print(SYSTEM_PROMPT)

You are a SQL query generator for snowflake.

DATABASE: AIRLINES.AIRLINES
SCHEMA: {
  "dialect": "snowflake",
  "database": "AIRLINES",
  "Schema": "AIRLINES",
  "tables": [
    {
      "name": "AIRCRAFTS_DATA",
      "column_names": [
        {
          "name": "aircraft_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "model",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "range",
          "type": "NUMBER(38,0)",
          "null?": "Y"
        }
      ],
      "sample_rows": [
        {
          "aircraft_code": "773",
          "model": "{\"en\": \"Boeing 777-300\", \"ru\": \"Боинг 777-300\"}",
          "range": 11100
        }
      ],
      "row_count": 9
    },
    {
      "name": "AIRPORTS_DATA",
      "column_names": [
        {
          "name": "airport_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": 

In [5]:
from lisette import *

chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[execute_query])

res = chat("How many flights are there?")
res

{"flight_count":33121}

<details>

- id: `chatcmpl-aefa789r2irv0a7v5h1f4k`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=11, prompt_tokens=2649, total_tokens=2660, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [7]:
chat.print_hist()

{'role': 'user', 'content': 'How many flights are there?'}

Message(content='', role='assistant', tool_calls=[{'function': {'arguments': '{"query":"SELECT COUNT(*) AS \\"flight_count\\" FROM AIRLINES.AIRLINES.\\"FLIGHTS\\""}', 'name': 'execute_query'}, 'id': '422353124', 'type': 'function'}], function_call=None, reasoning_content='Need count of FLIGHTS. Use SELECT COUNT(*) FROM AIRLINES.AIRLINES."FLIGHTS".', provider_specific_fields={'refusal': None, 'reasoning': 'Need count of FLIGHTS. Use SELECT COUNT(*) FROM AIRLINES.AIRLINES."FLIGHTS".', 'reasoning_content': 'Need count of FLIGHTS. Use SELECT COUNT(*) FROM AIRLINES.AIRLINES."FLIGHTS".'})

{'tool_call_id': '422353124', 'role': 'tool', 'name': 'execute_query', 'content': 'query=\'SELECT COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES."FLIGHTS"\' success=True data=[{\'flight_count\': 33121}] error=None row_count=1 execution_time=0.353029727935791'}

{'role': 'assistant', 'content': 'You have no more tool uses. Please summarize your 

In [8]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[execute_query])
res = chat("Which airport has the most departures?", max_steps=5)
res

The airport with the most departures is **DME** (Domodedovo International Airport), with **3,217** departures.

<details>

- id: `chatcmpl-rwg8d6wfyng5pcawk997rk`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=41, prompt_tokens=3121, total_tokens=3162, completion_tokens_details=None, prompt_tokens_details=None)`

</details>